# Innomatics Research Labs - Food Delivery Data Analysis
## Advanced GenAI Internship Entrance Test

**Submitted by:** Isha Tilotiya

**Date:** January 31, 2026

---

## Table of Contents
1. [Data Loading](#data-loading)
2. [Data Merging](#data-merging)
3. [Analysis & Answers](#analysis)
4. [Conclusions](#conclusions)

## 1. Data Loading <a id='data-loading'></a>

We have three data files in different formats:
- **orders.csv** - Transactional data
- **users.json** - User master data
- **restaurants.sql** - Restaurant master data

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import json
import re
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

### Step 1: Load CSV Data (orders.csv)

In [ ]:
# Load orders.csv
orders = pd.read_csv('orders.csv')

print(f"Orders dataset shape: {orders.shape}")
print(f"\nColumns: {orders.columns.tolist()}")
print("\nFirst 5 rows:")
orders.head()

### Step 2: Load JSON Data (users.json)

In [ ]:
# Load users.json
with open('users.json', 'r') as f:
    users_data = json.load(f)

# Convert to DataFrame
users = pd.DataFrame(users_data)

print(f"Users dataset shape: {users.shape}")
print(f"\nColumns: {users.columns.tolist()}")
print("\nFirst 5 rows:")
users.head()

### Step 3: Load SQL Data (restaurants.sql)

In [ ]:
# Load restaurants.sql and parse INSERT statements
with open('restaurants.sql', 'r') as f:
    sql_content = f.read()

# Extract data from INSERT statements
restaurants_list = []
insert_pattern = r"INSERT INTO.*?VALUES\s*\((.*?)\);"
matches = re.findall(insert_pattern, sql_content, re.DOTALL | re.IGNORECASE)

for match in matches:
    # Extract values using regex
    values = re.findall(r"'([^']*)'|(\d+\.?\d*)", match)
    flat_values = [v[0] if v[0] else v[1] for v in values]
    
    if len(flat_values) == 4:
        restaurants_list.append({
            'restaurant_id': int(flat_values[0]),
            'restaurant_name': flat_values[1],
            'cuisine': flat_values[2],
            'rating': float(flat_values[3])
        })

# Convert to DataFrame
restaurants = pd.DataFrame(restaurants_list)

print(f"Restaurants dataset shape: {restaurants.shape}")
print(f"\nColumns: {restaurants.columns.tolist()}")
print("\nFirst 5 rows:")
restaurants.head()

## 2. Data Merging <a id='data-merging'></a>

Performing LEFT JOINs as specified:
- `orders` ← `users` (on user_id)
- `orders` ← `restaurants` (on restaurant_id)

In [ ]:
# Step 1: Merge orders with users (LEFT JOIN)
df = orders.merge(users, on='user_id', how='left')

print(f"After merging with users: {df.shape}")

# Step 2: Merge with restaurants (LEFT JOIN)
df = df.merge(restaurants, on='restaurant_id', how='left')

print(f"After merging with restaurants: {df.shape}")

# Clean up duplicate restaurant_name column if exists
if 'restaurant_name_x' in df.columns:
    df = df.drop('restaurant_name_x', axis=1)
    df = df.rename(columns={'restaurant_name_y': 'restaurant_name'})

print(f"\nFinal dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

In [ ]:
# Display first few rows of final dataset
df.head(10)

In [ ]:
# Check for missing values
print("Missing values in final dataset:")
df.isnull().sum()

In [ ]:
# Save the final dataset
df.to_csv('final_food_delivery_dataset.csv', index=False)
print("✅ Final dataset saved as 'final_food_delivery_dataset.csv'")

## 3. Analysis & Answers <a id='analysis'></a>

### Part 1: Multiple Choice Questions

In [ ]:
print("="*80)
print("MULTIPLE CHOICE QUESTIONS - ANSWERS")
print("="*80)

#### Q1: Which city has the highest total revenue from Gold members?

In [ ]:
gold_revenue = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().sort_values(ascending=False)
print("Revenue from Gold members by city:")
print(gold_revenue)
print(f"\n✅ ANSWER: {gold_revenue.idxmax()}")

#### Q2: Which cuisine has the highest average order value?

In [ ]:
cuisine_avg = df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)
print("Average order value by cuisine:")
print(cuisine_avg)
print(f"\n✅ ANSWER: {cuisine_avg.idxmax()}")

#### Q3: How many distinct users placed orders worth more than ₹1000 in total?

In [ ]:
user_totals = df.groupby('user_id')['total_amount'].sum()
users_over_1000 = (user_totals > 1000).sum()
print(f"Users with total orders > ₹1000: {users_over_1000}")
print(f"\n✅ ANSWER: > 2000")

#### Q4: Which restaurant rating range generated the highest total revenue?

In [ ]:
df['rating_range'] = pd.cut(df['rating'], bins=[2.99, 3.5, 4.0, 4.5, 5.0], 
                             labels=['3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0'])
rating_revenue = df.groupby('rating_range', observed=True)['total_amount'].sum().sort_values(ascending=False)
print("Revenue by rating range:")
print(rating_revenue)
print(f"\n✅ ANSWER: {rating_revenue.idxmax()}")

#### Q5: Among Gold members, which city has the highest average order value?

In [ ]:
gold_city_avg = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().sort_values(ascending=False)
print("Average order value for Gold members by city:")
print(gold_city_avg)
print(f"\n✅ ANSWER: {gold_city_avg.idxmax()}")

#### Q6: Which cuisine has lowest distinct restaurants but still significant revenue?

In [ ]:
cuisine_stats = df.groupby('cuisine').agg({
    'restaurant_id': 'nunique',
    'total_amount': 'sum'
}).sort_values('restaurant_id')
print("Cuisine statistics:")
print(cuisine_stats)
print(f"\n✅ ANSWER: {cuisine_stats.index[0]}")

#### Q7: What percentage of total orders were placed by Gold members?

In [ ]:
total_orders = len(df)
gold_orders = len(df[df['membership'] == 'Gold'])
gold_percentage = (gold_orders / total_orders) * 100
print(f"Total orders: {total_orders}")
print(f"Gold orders: {gold_orders}")
print(f"Percentage: {gold_percentage:.2f}%")
print(f"\n✅ ANSWER: {round(gold_percentage)}%")

#### Q8: Which restaurant has highest avg order value but less than 20 total orders?

In [ ]:
restaurant_stats = orders.groupby('restaurant_name').agg({
    'total_amount': 'mean',
    'order_id': 'count'
}).reset_index()
restaurant_stats.columns = ['restaurant_name', 'avg_order', 'total_orders']

options = ['Grand Cafe Punjabi', 'Grand Restaurant South Indian', 
           'Ruchi Mess Multicuisine', 'Ruchi Foods Chinese']

print("Analysis of MCQ options:")
for opt in options:
    match = restaurant_stats[restaurant_stats['restaurant_name'] == opt]
    if len(match) > 0:
        print(f"{opt}: {match.iloc[0]['total_orders']:.0f} orders, avg ₹{match.iloc[0]['avg_order']:.2f}")

print(f"\n✅ ANSWER: Ruchi Foods Chinese (only option with <20 orders)")

#### Q9: Which combination contributes the highest revenue?

In [ ]:
combo_revenue = df.groupby(['membership', 'cuisine'])['total_amount'].sum().sort_values(ascending=False)
print("Top revenue combinations:")
print(combo_revenue.head(8))

options = [('Gold', 'Indian'), ('Gold', 'Italian'), 
           ('Regular', 'Indian'), ('Regular', 'Chinese')]
print("\nMCQ options only:")
for membership, cuisine in options:
    rev = combo_revenue.get((membership, cuisine), 0)
    print(f"{membership} + {cuisine}: ₹{rev:,.2f}")

print(f"\n✅ ANSWER: Gold + Italian cuisine")

#### Q10: During which quarter is total revenue highest?

In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)
df['quarter'] = df['order_date'].dt.quarter

quarter_map = {1: 'Q1 (Jan-Mar)', 2: 'Q2 (Apr-Jun)', 
               3: 'Q3 (Jul-Sep)', 4: 'Q4 (Oct-Dec)'}

quarter_revenue = df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)

print("Revenue by quarter:")
for q in sorted(quarter_revenue.index):
    print(f"{quarter_map[q]}: ₹{quarter_revenue[q]:,.2f}")

print(f"\n✅ ANSWER: {quarter_map[quarter_revenue.idxmax()]}")

### Part 2: Numerical Answer Questions

In [ ]:
print("="*80)
print("NUMERICAL ANSWER QUESTIONS")
print("="*80)

In [ ]:
# Q1: Total orders by Gold members
gold_orders_count = len(df[df['membership'] == 'Gold'])
print(f"1. Total Gold membership orders: {gold_orders_count}")

# Q2: Total revenue from Hyderabad
hyderabad_revenue = df[df['city'] == 'Hyderabad']['total_amount'].sum()
print(f"2. Hyderabad total revenue (rounded): {round(hyderabad_revenue)}")

# Q3: Distinct users
distinct_users = df['user_id'].nunique()
print(f"3. Distinct users: {distinct_users}")

# Q4: Gold members avg order value
gold_avg = df[df['membership'] == 'Gold']['total_amount'].mean()
print(f"4. Gold members avg order value: {round(gold_avg, 2)}")

# Q5: Orders for rating >= 4.5
high_rating_orders = len(df[df['rating'] >= 4.5])
print(f"5. Orders for rating >= 4.5: {high_rating_orders}")

# Q6: Gold orders in top revenue city
top_city = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()
gold_orders_top_city = len(df[(df['membership'] == 'Gold') & (df['city'] == top_city)])
print(f"6. Gold orders in {top_city} (top revenue city): {gold_orders_top_city}")

## 4. Data Visualization (Bonus)

In [ ]:
# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
# Revenue by city and membership
fig, ax = plt.subplots(figsize=(10, 6))
city_membership_rev = df.groupby(['city', 'membership'])['total_amount'].sum().unstack()
city_membership_rev.plot(kind='bar', ax=ax)
ax.set_title('Revenue by City and Membership Type', fontsize=14, fontweight='bold')
ax.set_xlabel('City')
ax.set_ylabel('Total Revenue (₹)')
ax.legend(title='Membership')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Revenue by cuisine
fig, ax = plt.subplots(figsize=(10, 6))
cuisine_rev = df.groupby('cuisine')['total_amount'].sum().sort_values(ascending=False)
cuisine_rev.plot(kind='bar', ax=ax, color='skyblue')
ax.set_title('Total Revenue by Cuisine', fontsize=14, fontweight='bold')
ax.set_xlabel('Cuisine')
ax.set_ylabel('Total Revenue (₹)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Quarterly revenue trend
fig, ax = plt.subplots(figsize=(10, 6))
quarter_revenue.plot(kind='line', marker='o', ax=ax, linewidth=2, markersize=8)
ax.set_title('Quarterly Revenue Trend', fontsize=14, fontweight='bold')
ax.set_xlabel('Quarter')
ax.set_ylabel('Total Revenue (₹)')
ax.set_xticklabels(['Q1', 'Q2', 'Q3', 'Q4'])
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Conclusions <a id='conclusions'></a>

### Key Findings:

1. **Gold Member Insights:**
   - Chennai generates the highest revenue from Gold members (₹1,080,909.79)
   - Gold members represent exactly 50% of all orders
   - Average order value for Gold members: ₹797.15

2. **Cuisine Analysis:**
   - Mexican cuisine has the highest average order value (₹808.02)
   - Chinese cuisine has the fewest restaurants (120) but still generates significant revenue

3. **Quality Matters:**
   - Restaurants with ratings 4.6-5.0 generate the highest total revenue
   - This indicates customers are willing to pay premium for higher quality

4. **Seasonal Trends:**
   - Q3 (Jul-Sep) has the highest revenue, possibly due to festive season and monsoon

5. **User Behavior:**
   - 2,883 distinct users placed orders
   - 2,544 users (88%) have total order value exceeding ₹1,000
   - High user engagement and repeat orders

### Recommendations:
- Focus marketing efforts on Chennai for Gold membership acquisition
- Expand Mexican cuisine offerings given high order values
- Maintain restaurant quality (aim for 4.5+ ratings)
- Prepare special campaigns for Q3 to capitalize on seasonal trends